In [1]:
import pandas as pd
import json

In [2]:
user_input = ["간장", "후춧가루", "밥", "아귀", "콩나물", "두부", "고춧가루", "소금", "계란"]

In [6]:
def recmd_menu(user_input_list):
    first_df = pd.read_csv("tf_idf_df.csv")
    
    result_dict = {"Menu": [],
                   "Score": [],
                   "Same_Ingre": [],
                   "Same_Ingre_len": []}

    for idx in range(537):
        max_menu = ""
        max_score = 0
        max_list = []

        for menu in first_df["RECIPE_NM_KO"].drop_duplicates():
            each_score = 0
            avg = 0
            same_list = []

            target_df = first_df[first_df["RECIPE_NM_KO"] == menu].reset_index(drop = True)
            target_df_sum = target_df["SCORE"].sum()
            must_list = list(target_df[target_df["SCORE"] == 5]["Ingredient"])

            check_bool = True

            for must in must_list:
                chk_bool = must in user_input
                check_bool = check_bool and chk_bool

            if not check_bool:
                continue

            for i, compare in enumerate(target_df["Ingredient"]):
                for user_v in user_input_list:
                    if compare == user_v:
                        each_score = each_score + target_df["SCORE"][i]
                        same_list.append(user_v)
                        break

            avg = each_score / target_df_sum

            if max_score < avg:
                max_score = avg
                max_menu = menu
                max_list = same_list

        if max_menu == "":
            break

        result_dict["Menu"].append(max_menu)
        result_dict["Score"].append(round(max_score * 100, 2))
        result_dict["Same_Ingre"].append(max_list)
        result_dict["Same_Ingre_len"].append(len(max_list))

        drop_index = first_df[first_df["RECIPE_NM_KO"] == max_menu].index
        first_df = first_df.drop(drop_index).reset_index(drop=True)
        result_df = pd.DataFrame(result_dict).sort_values(by=['Same_Ingre_len', 'Score'], ascending=False).reset_index(drop=True)
        final_dict = result_df.to_dict('index')
        final_json = json.dumps(final_dict, ensure_ascii=False)
        
    return final_json